In [1]:
import numpy as np
from enum import Enum

# Global variables representing the different cell states
EMPTY = 0 
O = 1 
X = 2 

Win_vale = 1.0 
Draw_vale = 0.0
Loss_vale = 0.0

# Enumeration class for game results
class PossibleOutcomes(Enum):
    NOT_FINISHED = 0
    O_Win = 1
    X_Win = 2
    DRAW = 3

# Board dimensions and size of our board.
Cols = 3
Rows = 3
BoardSize = Cols * Rows
#-------------------------------------------------------------------------------------------------------
"""
Code for Class: Board
which contains several crucial methods inorder for the game to work and function correctly.
Such as: the games state, move, check for wins and other important things.
"""
#-------------------------------------------------------------------------------------------------------

"""
Win_check_directions is a dic contaning ALL possible directions in which a winning move can occur!
"""

# Our Board Class contains all states
class Board:
    # Check directions for winning moves
    WIN_CHECK_DIRS = {0: [(1, 1), (1, 0), (0, 1)],
                      1: [(1, 0)],
                      2: [(1, 0), (1, -1)],
                      3: [(0, 1)],
                      6: [(0, 1)]}
    
#-------------------------------------------------------------------------------------------------------
    def hash_value(self) :
        """
        - Hash_val method returns a unique int representing the current state of the board
        - Done through iterating ovver the sate of the board, multiplying the current result by 3.
        - Then adding the current state of the cell
        """
        res = 0
        for i in range(BoardSize):
            res = res * 3 + self.state[i]
        return res

#-------------------------------------------------------------------------------------------------------
    def other_who(who):
        """
        - other_who method, takes in a symbol etiher X or O. and returns the opposite.
        """
        if who == X:
            return O
        
        if who == O:
            return X
        
#-------------------------------------------------------------------------------------------------------
    def __init__(self, s=None):
        """
        - __init__ method is the constructor of it all
        - takes in an OPTIONAL paramter s, which is a state that the board can be initialized with
        - HOWEVER! if no state is passed in. The board is initialized with an EMPTY state.
        """
        if s is None:
            self.state = np.ndarray(shape=(1, BoardSize), dtype=int)[0]
            self.reset()
        else:
            self.state = s.copy()
            
#-------------------------------------------------------------------------------------------------------
    def reset(self):
        """
        Sets all the cells in the board to the empty state
        """
        self.state.fill(EMPTY)
        
#-------------------------------------------------------------------------------------------------------
    def num_empty(self):
        """
        - num empty method returns the number of empty cells in the board.
        """
        return np.count_nonzero(self.state == EMPTY)
    
#-------------------------------------------------------------------------------------------------------
    def random_empty_spot(self):
        """
        Returns a random empty cell on the board, done through a loop.
        """
        index = np.random.randint(self.num_empty())
        for i in range(9):
            if self.state[i] == EMPTY:
                if index == 0:
                    return i
                else:
                    index = index - 1
                    
#-------------------------------------------------------------------------------------------------------
    def is_legal(self, pos):
        """
        - Is_legal method checks wheter or not if a move to a position is "legal"
        - by checking the pos in question is empty.
        """
        return pos in range(BoardSize) and self.state[pos] == EMPTY
    
#-------------------------------------------------------------------------------------------------------
    def move(self, position, who) -> (np.ndarray, PossibleOutcomes, bool):
        """
        - move method. Makes a move to a position on the board, with the symbols X or O.
        - Checks if the move is LEGAL and if the move causes a WIN or DRAW
        """
        if self.state[position] != EMPTY:
            print('Illegal move')
        self.state[position] = who
        if self.check_win():
            return self.state, PossibleOutcomes.X_Win if who == X else PossibleOutcomes.O_Win, True

        if self.num_empty() == 0:
            return self.state, PossibleOutcomes.DRAW, True

        return self.state, PossibleOutcomes.NOT_FINISHED, False
    
#-------------------------------------------------------------------------------------------------------
    def apply_direction(self, pos, direction: (int, int)):
        """
        - apply_direction method takes a position and direction as a tuple.
        - and returns a new position after moving in THAT PARTICULAR direction.
        """
        row = pos // 3
        col = pos % 3
        row += direction[0]
        if row < 0 or row > 2:
            return -1
        col += direction[1]
        if col < 0 or col > 2:
            return -1

        return row * 3 + col
#-------------------------------------------------------------------------------------------------------
    def check_win_in_dir(self, pos, direction: (int, int)):
        """
        - Check_win dir takes a position & direction and checks wheter or not if theres a win in that direction
        - Done by checking current cell in the given direction if the symbols are the same.
        """
        c = self.state[pos]
        if c == EMPTY:
            return False

        p1 = int(self.apply_direction(pos, direction))
        p2 = int(self.apply_direction(p1, direction))

        if p1 == -1 or p2 == -1:
            return False

        if c == self.state[p1] and c == self.state[p2]:
            return True

        return False
    
#-------------------------------------------------------------------------------------------------------
    def who_won(self):
        """
        - who_won as it says by itself returns the symbol of the player who has won the game
        - And none if the game has not been won.
        """
        for start_pos in self.WIN_CHECK_DIRS:
            if self.state[start_pos] != EMPTY:
                for direction in self.WIN_CHECK_DIRS[start_pos]:
                    res = self.check_win_in_dir(start_pos, direction)
                    if res:
                        return self.state[start_pos]
        return EMPTY
#-------------------------------------------------------------------------------------------------------

    def check_win(self):
        """
        - Check win: method iterates through each possible direction in the Win_check_dirs we declared in the beginning.
        - and calls the check_win_dir to detect IF theres a win! TRUE if WIN. FALSE if NOT
        """
        for start_pos in self.WIN_CHECK_DIRS:
            if self.state[start_pos] != EMPTY:
                for direction in self.WIN_CHECK_DIRS[start_pos]:
                    res = self.check_win_in_dir(start_pos, direction)
                    if res:
                        return True
        return False


In [2]:
"""
- Imports abstrakt class ABS från abs.
- & Uses it to define abstrakt player class
- Class has has 3 abstrakt classes: Move, Final & New game.
- These 3 methods must be implemented by a subclass so the class can be used.
"""

from abc import ABC, abstractmethod

class Player(ABC):
    def __init__(self):
        super().__init__()
#-------------------------------------------------------------------------------------------------------

    def move(self, board: Board) -> (PossibleOutcomes, bool):
        pass
#-------------------------------------------------------------------------------------------------------

    def final_result(self, result: PossibleOutcomes):
        pass
#-------------------------------------------------------------------------------------------------------

    def new_game(self, who: int):
        pass

In [3]:
from typing import Dict, List
import numpy as np

"""
This code is a single-player implementation for a Tic Tac Toe game, 
where the player uses Q-learning to choose his moves.
"""


class QLearn(Player):
    def __init__(self, alpha=0.05, gamma=0.95, q_init=0.6):
        """
        The QLearn class inherits from another class called PLAYER and has four properties: 
        - who
        - q
        - move_list
        - learning_rate
        Each of them is created and initialized in the init method.
        """

        self.who = None             # variable to store the player's piece (X or O)
        self.q = {}                 # dictionary to store Q-values for each board state
        self.move_list = []         # list to store the player's moves in the current game
        self.learning_rate = alpha  # learning rate for updating Q-values
        self.value_discount = gamma # discount factor for future rewards
        self.q_init_val = q_init    # initial value for Q-values of new states
        super().__init__()          # call the parent class's __init__ method
#-------------------------------------------------------------------------------------------------------
    def get_q(self, board_hash: int) -> np.ndarray:
        # The get_q method takes an argument: board_hash and returns a numpy array.
        
         # Checks if Q values for that state have been calculated before by checking if board_hash exists in dic q
        if board_hash in self.q:
            qvals = self.q[board_hash]
        else:
            
            # If not, initialize Q-values for all possible actions
            qvals = np.full(9, self.q_init_val)
            self.q[board_hash] = qvals
        return qvals
    
#-------------------------------------------------------------------------------------------------------
    def get_move(self, board: Board):
        """
        - The get_move method takes in an argument: board, Board.
        - It uses the hash_value method to calculate the hash value of the current board state 
        - Calls the get_q method to get the Q values of the current state
        - Selects the action with the highest Q value. If the action is not legal, then its Q value is set to -1.
        """
        board_hash = board.hash_value()   # calculate the hash of the current board state
        qvals = self.get_q(board_hash)    # get the Q-values for the current state
        while True:
            m = np.argmax(qvals)          # select the action with the highest Q-value
                                          # Check if the selected action is legal
            if board.is_legal(m):
                return m
            else:
                qvals[m] = -1.0           # set the Q-value of an illegal action to -1
                
#-------------------------------------------------------------------------------------------------------
    def move(self, board: Board):
        """
        - The move method uses the get_move method to select the next move and adds it to move_list.
        - Then it makes the move on the board and returns the result and whether the game is over or not.
        """
        m = self.get_move(board) # select the next move
        self.move_list.append((board.hash_value(), m))  # add the move to the move history
        _, res, finished = board.move(m, self.who)  # make the move on the board
        return res, finished
    
#-------------------------------------------------------------------------------------------------------
    # Updates Q-vals based on the final result of the game.
    def final_result(self, result: PossibleOutcomes):
        """
        - The final_result method takes an argument, result, Type: PossibleOutcomes.
        - Uses it to update the Q values based on the final result of the game.
        - Reverses move_list to update the Q value for each move based on the next moves highest q value.
        """
        
        if (result == PossibleOutcomes.O_Win and self.who == O) or (
                result == PossibleOutcomes.X_Win and self.who == X):
            final_value = Win_vale 
            
        elif (result == PossibleOutcomes.O_Win and self.who == X) or (
                result == PossibleOutcomes.X_Win and self.who == O):
            final_value = Loss_vale  
            
        elif result == PossibleOutcomes.DRAW:
            final_value = Draw_vale  


        self.move_list.reverse()
        next_max = -1.0  # variable to store the maximum Q-value of the next state

        for h in self.move_list:
            qvals = self.get_q(h[0])
            if next_max < 0:  # First time through the loop
                qvals[h[1]] = final_value
            else:
                qvals[h[1]] = qvals[h[1]] * (
                            1.0 - self.learning_rate) + self.learning_rate * self.value_discount * next_max

            next_max = max(qvals)
            
#-------------------------------------------------------------------------------------------------------
    def new_game(self, who):
        """
        - The new_game method takes in argument: who, and uses it to start a new game.
        - Sets the value of who to the incoming argument and clears move_list.
        """
        self.who = who
        self.move_list = []


In [4]:
class RandomPlayer(Player):
    def __init__(self):
        self.who = None  # initiera spelarens sida till None
        super().__init__()  # anropa föräldraklassens __init__ metod
        
#-------------------------------------------------------------------------------------------------------
    def move(self, board: Board) -> (PossibleOutcomes, bool):
        """
        - This Method is called when its the Random Players turn.
        - Takes in BOARD as parameter. Calls MOVE method för BOARD-OBJEKT and sends a random empty position, and the players who.
        - Metod returns the result and IF the game is Finished T/F
        """
        _, res, finished = board.move(board.random_empty_spot(), self.who)
        return res, finished  
    
    
#-------------------------------------------------------------------------------------------------------
    def new_game(self, who: int):
        """
        - Denna metod anropas när ett nytt spel börjar. Den tar emot en 'who' integer som
        - representerar spelarens sida och tilldelar den till 'who' attributet för spelaren.
        """
        self.who = who



In [5]:
"""
- Create a list: Player with player1 & player2
- Calls new_game with X & O
- Calls reset.board to reset the game
- Defins VAR finished as FALSE, Wheter or not the game is done.

- While-loop as long as finished is FALSE.
- Calls MOVE and assigns the result and finished VARS
- IF Finished == TRUE then final_result is set to either DRAW or X_Win

- Calls MOVE for O player and assigsn the result and finished VARS
- IF Finished == TRUE then final_result is set to either DRAW or O_Win

- Run a loop for each player in the players list and call the final result func with final result as param.
"""

def play_game(board: Board, player1: Player, player2: Player):
    players = [player1, player2]
    player1.new_game(X)
    player2.new_game(O)
    board.reset()

    finished = False
    while not finished:
        result, finished = players[0].move(board)
        if finished:
            final_result = PossibleOutcomes.DRAW if result == PossibleOutcomes.DRAW else PossibleOutcomes.X_Win
            break
            
            
        result, finished = players[1].move(board)
        if finished:
            final_result = PossibleOutcomes.DRAW if result == PossibleOutcomes.DRAW else PossibleOutcomes.O_Win
            break

    for player in players:
        player.final_result(final_result)
    return final_result


In [6]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [7]:
number_of_games = 10000

def play(player1: Player, player2: Player):
    board = Board()
    draw_count = 0
    X_count = 0
    O_count = 0
    for _ in range(number_of_games):
        result = play_game(board, player1, player2)
        if result == PossibleOutcomes.X_Win:
            X_count += 1
        elif result == PossibleOutcomes.O_Win:
            O_count += 1
        else:
            draw_count += 1
    print("-------------------------------------------------------------------------------------------")
    print("After {} games we have the following: \nDraws: {}. \nPlayer 1 wins: {}, \nPlayer 2 wins: {}.".format(number_of_games, draw_count,
                                                                                         X_count, O_count))
    print("-------------------------------------------------------------------------------------------")
    print("Draws: {:.2%}\nPlayer 1 wins: {:.2%}\nPlayer 2 wins: {:.2%}".format(
        draw_count / number_of_games, X_count / number_of_games, O_count / number_of_games))
    print("-------------------------------------------------------------------------------------------")

In [8]:
print("Random VS Random")
play(RandomPlayer(), RandomPlayer())
print("Random VS Random")

Random VS Random
-------------------------------------------------------------------------------------------
After 10000 games we have the following: 
Draws: 1287. 
Player 1 wins: 5895, 
Player 2 wins: 2818.
-------------------------------------------------------------------------------------------
Draws: 12.87%
Player 1 wins: 58.95%
Player 2 wins: 28.18%
-------------------------------------------------------------------------------------------
Random VS Random


In [9]:
play(QLearn(), RandomPlayer())

-------------------------------------------------------------------------------------------
After 10000 games we have the following: 
Draws: 352. 
Player 1 wins: 9323, 
Player 2 wins: 325.
-------------------------------------------------------------------------------------------
Draws: 3.52%
Player 1 wins: 93.23%
Player 2 wins: 3.25%
-------------------------------------------------------------------------------------------


In [10]:
play(QLearn(), RandomPlayer())

-------------------------------------------------------------------------------------------
After 10000 games we have the following: 
Draws: 175. 
Player 1 wins: 9737, 
Player 2 wins: 88.
-------------------------------------------------------------------------------------------
Draws: 1.75%
Player 1 wins: 97.37%
Player 2 wins: 0.88%
-------------------------------------------------------------------------------------------


In [11]:
play(QLearn(), RandomPlayer())

-------------------------------------------------------------------------------------------
After 10000 games we have the following: 
Draws: 241. 
Player 1 wins: 9616, 
Player 2 wins: 143.
-------------------------------------------------------------------------------------------
Draws: 2.41%
Player 1 wins: 96.16%
Player 2 wins: 1.43%
-------------------------------------------------------------------------------------------


In [12]:
play(QLearn(), RandomPlayer())

-------------------------------------------------------------------------------------------
After 10000 games we have the following: 
Draws: 212. 
Player 1 wins: 9625, 
Player 2 wins: 163.
-------------------------------------------------------------------------------------------
Draws: 2.12%
Player 1 wins: 96.25%
Player 2 wins: 1.63%
-------------------------------------------------------------------------------------------


In [13]:
play(QLearn(), RandomPlayer())

-------------------------------------------------------------------------------------------
After 10000 games we have the following: 
Draws: 233. 
Player 1 wins: 9579, 
Player 2 wins: 188.
-------------------------------------------------------------------------------------------
Draws: 2.33%
Player 1 wins: 95.79%
Player 2 wins: 1.88%
-------------------------------------------------------------------------------------------


In [14]:
play(QLearn(), RandomPlayer())

-------------------------------------------------------------------------------------------
After 10000 games we have the following: 
Draws: 249. 
Player 1 wins: 9499, 
Player 2 wins: 252.
-------------------------------------------------------------------------------------------
Draws: 2.49%
Player 1 wins: 94.99%
Player 2 wins: 2.52%
-------------------------------------------------------------------------------------------
